In [14]:
import numpy as np

In [45]:
def write_header_lines(file, layer_height, line_width, layer_count, mesh, start_x, start_y, start_z=0.3):
    with open(file, 'w') as f:
        header_lines = [
            f";layer_height = {layer_height}",
            f"\n;line_width = {line_width}",
            f"\n;layer_count = {layer_count}",
            f"\n;mesh = {mesh}"
        ]
        
        initialize_lines = [
            "\nG21 ;start of the code",
            "\nG1 Z15 F300",
            "\nG28 X0 Y0 ;Home",
            "\nG92 X0 Y0 ;Consider this as current",
            "\nG0 X50 Y50 F3000 ;Go-to Offset",  
            "\nG92 X0 Y0 ;Reset",
            "\n",
            f"\nG0 F3600 X{start_x:.3f} Y{start_y:.3f} Z{start_z:.3f} ;Go to start position",
            "\nM7",
            "\n\n"
        ]
        
        f.writelines(header_lines)
        f.writelines(initialize_lines)
        
        
def writ_finish_lines(file):
    with open(file, 'a') as f:
        finish_lines = [
            "\n\n;Finish",
            "\nM9",
            "\nG1 Z10.000"
            "\nG28 X-100 Y-100;Home",
        ]
        f.writelines(finish_lines)

In [50]:
def write_G1_line(delta_x, delta_y, xy, lines, pause=True):
    xy += [delta_x, delta_y]
    lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f}")
    if pause:
        lines.append(f"\nG4 P30")
    # print(xy)
    return xy, lines

def write_init_layer(xy, z, lines):
    lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f} Z{z:.3f}")
    return lines    

In [36]:
def draw_layer_out_in(file, xy1, xy3, z, w):
    
    xy = np.array([xy1[0] + w/2, xy1[1] + w/2])
    a = xy3[0] - xy1[0] - w
    b = xy3[1] - xy1[1] - w
    
    lines = []
    lines = write_init_layer(xy, z, lines)
    for i in range (np.min([int(a/w), int(b/w)])+1):
        if (i%2==0):
            xy, lines = write_G1_line(a-w*i, 0, xy, lines)
            xy, lines = write_G1_line(0, b-w*i, xy, lines)
        else:
            xy, lines = write_G1_line(-a+w*(i-1), 0, xy, lines)
            xy, lines = write_G1_line(0, -b+w*(i-1), xy, lines, pause=False)
            xy, lines = write_G1_line(w, w, xy, lines, pause=False)
    
    with open(file, 'a') as f:
        f.writelines(lines)
        
        
def draw_layer_in_out(file, xy1, xy3, z, w):
    
    xy = np.array([xy1[0] + w/2, xy1[1] + w/2])
    a = xy3[0] - xy1[0] - w
    b = xy3[1] - xy1[1] - w
    
    lines_inverse = []
    lines_inverse = write_init_layer(xy, z, lines_inverse)
    for i in range (np.min([int(a/w), int(b/w)])+1):
        if (i%2==0):
            xy, lines_inverse = write_G1_line(a-w*i, 0, xy, lines_inverse)
            xy, lines_inverse = write_G1_line(0, b-w*i, xy, lines_inverse)
        else:
            xy, lines_inverse = write_G1_line(-a+w*(i-1), 0, xy, lines_inverse)
            xy, lines_inverse = write_G1_line(0, -b+w*(i-1), xy, lines_inverse, pause=False)
            xy, lines_inverse = write_G1_line(w, w, xy, lines_inverse)
            
    lines = []
    lines = write_init_layer(xy, z, lines)
    l = len(lines_inverse)
    for i in range (len(lines_inverse)):
        if (lines_inverse[i].startswith('G4')):
            lines.append(lines_inverse[l-i-2])
            lines.append(lines_inverse[l-i-1])
            i += 1
        else:
            lines.append(lines_inverse[l-i-1])
            
    with open(file, 'a') as f:
        f.writelines(lines)
        
        

def draw_layer(file, xy1, xy3, z, w, direction):
    """Draw single platform layer by concentrate infill

    Args:
        xy1, xy3 (ndarray)
        w (float)          : line width
        direction (int)    : 0 for out -> in (counter clockwise) 
                             1 for in -> out (clockwise)
    """
    
    if direction == 0:
        draw_layer_out_in(file, xy1, xy3, z, w)
    elif direction == 1:
        draw_layer_in_out(file, xy1, xy3, z, w)

In [39]:
layer_height = 0.3
line_width = 0.4
layer_count = 2
mesh = 'side'
length_a = 20
length_b = 30
wall_layer_count = 0
file = '7.23/side_'+str(length_a)+'_'+str(length_b)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

# xb_list = [np.sqrt(4*(layer_height/2+i*layer_height)) for i in range (10)]

for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-length_a/2 + xb, -length_b/2])
    xy3 = np.array([length_a/2, length_b/2])
    
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, True)
        
writ_finish_lines(file)

In [42]:
layer_height = 0.3
line_width = 0.4
layer_count = 2
mesh = 'middle'
length_a = 8.6
length_b = 13.5
wall_layer_count = 0
file = '7.23/middle_'+str(length_b)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

# xb_list = [np.sqrt(4*(layer_height/2+i*layer_height)) for i in range (10)]

for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-length_a/2 + xb, -length_b/2])
    xy3 = np.array([length_a/2 - xb, length_b/2])
    
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, True)
        
writ_finish_lines(file)

In [53]:
layer_height = 0.4
line_width = 0.4
layer_count = 2
mesh = 'base'

x1 = 4.3
x2 = 4.6
x3 = 15

y1 = 1
y2 = 15

file = '7.24/base_'+str(x3)+'_'+str(y2)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -x3, -y2)

# base film (2 layers):
xy1 = np.array([-x3, -y2])
xy3 = np.array([x3, y2])
draw_layer(file, xy1, xy3, 0.3, line_width, False)
draw_layer(file, xy1, xy3, 0.7, line_width, True)

# middle part low:
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-x1+xb, -y2])
    xy3 = np.array([x1-xb, -y1])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
lines = [
    f"\nM9"
    f"\nG1 Z10.000"
    f"\nG1 X{-x1:.3f} Y{y1:.3f}"
    f"\nG1 Z1.1"
    f"\nM7"
]        

# middle part high:
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-x1+xb, y1])
    xy3 = np.array([x1-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
lines = [
    f"\nM9"
    f"\nG1 Z10.000"
    f"\nG1 X{-x3:.3f} Y{-y2:.3f}"
    f"\nG1 Z1.1"
    f"\nM7"
]
with open(file, 'a') as f:
    f.writelines(lines)

# left side:
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-x3, -y2])
    xy3 = np.array([-x2-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
lines = [
    f"\nM9"
    f"\nG1 Z10.000"
    f"\nG1 X{x2:.3f} Y{-y2:.3f}"
    f"\nG1 Z1.1"
    f"\nM7"
]
with open(file, 'a') as f:
    f.writelines(lines)
        
# right side:
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([x2+xb, -y2])
    xy3 = np.array([x3, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
             
        
writ_finish_lines(file)

In [54]:
np.sqrt(4*(layer_height/2+9*layer_height))

3.898717737923586

In [55]:
3.9+4.6

8.5

In [56]:
4.3-3.9

0.3999999999999999